In [73]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

In [74]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [75]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [76]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [77]:
# add a custom layer
class CustomDense(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()

        self.weights = nn.Parameter(
            torch.Tensor(size_out, size_in)
        )  
        # nn.Parameter - trainable parameters, it's what we send to the constructor of the Optimizier.
        # serach for model.parameters() 

        self.bias = nn.Parameter(
            torch.Tensor(size_out)
        ) 

        # initialize weights and biases

        # He initialization
        nn.init.kaiming_uniform_(self.weights, a=np.sqrt(5))
        nn.init.uniform_(self.bias, -0.1, 0.1)

        # send to device
        self.weights.to(device)
        self.bias.to(device)

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b

In [78]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            CustomDense(28*28, 512), # instead of nn.Linear
            nn.ReLU(),
            CustomDense(512, 512),
            nn.ReLU(),
            CustomDense(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

#model = torch.compile(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): CustomDense()
    (1): ReLU()
    (2): CustomDense()
    (3): ReLU()
    (4): CustomDense()
  )
)


In [79]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [80]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    # make it faster, no gradient computation
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [86]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 1.144160  [   64/60000]
loss: 1.169446  [ 6464/60000]
loss: 1.012073  [12864/60000]
loss: 1.147528  [19264/60000]
loss: 1.039977  [25664/60000]
loss: 1.077536  [32064/60000]
loss: 1.120531  [38464/60000]
loss: 1.079735  [44864/60000]
loss: 1.130474  [51264/60000]
loss: 1.068897  [57664/60000]
tensor([[-1.9233, -2.8560, -1.1080, -2.0403, -1.0706,  2.5239, -1.1972,  2.8083,
          1.8150,  3.1091],
        [ 1.0765, -3.0039,  3.7151, -0.9506,  3.1667, -2.1034,  2.9084, -3.9011,
          1.4608, -1.9376],
        [ 1.9230,  4.7434, -0.4187,  3.4251,  0.7151, -2.9246,  0.4929, -3.8058,
         -1.8142, -3.0957],
        [ 1.4268,  3.7172, -0.3614,  2.6461,  0.4075, -2.0686,  0.2988, -2.8562,
         -1.5408, -2.2123],
        [ 1.0056, -1.3302,  1.4528, -0.1688,  1.3243, -0.9754,  1.3695, -2.0622,
          0.6725, -1.0309],
        [ 2.0949,  3.3704,  0.1627,  2.6770,  1.0273, -2.6558,  0.9006, -3.6517,
         -1.4901, -2.8980],
      

In [82]:

# save
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# load
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [85]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]



model.eval()
x, y = test_data[0][0], test_data[0][1]

# also with no autograd
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print("Probabilities: ", nn.Softmax(dim=1)(pred))

    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')



Probabilities:  tensor([[0.0025, 0.0010, 0.0056, 0.0022, 0.0058, 0.2118, 0.0051, 0.2815, 0.1042,
         0.3803]], device='mps:0')
Predicted: "Ankle boot", Actual: "Ankle boot"
